In [41]:
import pandas as pd
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


In [43]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Create Driver

In [5]:
driver = webdriver.Chrome(ChromeDriverManager().install())

page_url = "https://evehiclesdb.com/category/transport/electric-cars/"
driver.get(page_url)

C:\Users\kagom\AppData\Local\Temp\ipykernel_5416\1132155773.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


### Find EV Vehicles from Database

In [6]:
from selenium.webdriver.common.by import By

ev_categories = driver.find_elements(By.CLASS_NAME, "brand_list_item")

In [8]:
ev_categories[0].get_attribute('href')

'https://evehiclesdb.com/brands/aiways?category=electric-cars'

In [15]:
ev_element = driver.find_elements(By.CLASS_NAME, "top_cars_list_item")

In [16]:
ev_element

[<selenium.webdriver.remote.webelement.WebElement (session="9327e42da3dd67b085338629a19eb975", element="8434163d-aa15-4a25-9744-a58dc5479de8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9327e42da3dd67b085338629a19eb975", element="6bd27c8d-2a2f-4b88-9264-c390f52b09de")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9327e42da3dd67b085338629a19eb975", element="ece836ec-c0f2-48bd-b704-90563b62b425")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9327e42da3dd67b085338629a19eb975", element="033bb8a8-ec7d-4fae-960e-6320d5a6a7b5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9327e42da3dd67b085338629a19eb975", element="1745739a-cce5-4cab-b8a8-7f487ff23bc2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9327e42da3dd67b085338629a19eb975", element="aa735936-9a51-440a-b365-c40307c1b940")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9327e42da3dd67b085338629a19eb975", element="ab23b20f-42c3-4bd1-88da-f3

# Full Code

In [17]:
# Create Driver
driver = webdriver.Chrome(ChromeDriverManager().install())

#Go to the EVs DB page
page_url = "https://evehiclesdb.com/category/transport/electric-cars/"
driver.get(page_url)

#Find EV makes
ev_make = driver.find_elements(By.CLASS_NAME, "brand_list_item")

ev_makes = [] #creates an empty list
for make in ev_make:
    make_url = make.get_attribute('href')
    make_name = make.text
    ev_makes.append({'ev make' : make_name, 'url': make_url})

C:\Users\kagom\AppData\Local\Temp\ipykernel_5416\3269401552.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [18]:
ev_makes

[{'ev make': 'Aiways (1)',
  'url': 'https://evehiclesdb.com/brands/aiways?category=electric-cars'},
 {'ev make': 'Aston Martin (1)',
  'url': 'https://evehiclesdb.com/brands/aston-martin?category=electric-cars'},
 {'ev make': 'Audi (13)',
  'url': 'https://evehiclesdb.com/brands/audi?category=electric-cars'},
 {'ev make': 'BAIC (1)',
  'url': 'https://evehiclesdb.com/brands/baic?category=electric-cars'},
 {'ev make': 'BMW (9)',
  'url': 'https://evehiclesdb.com/brands/bmw?category=electric-cars'},
 {'ev make': 'Byton (3)',
  'url': 'https://evehiclesdb.com/brands/byton?category=electric-cars'},
 {'ev make': 'Citroen (8)',
  'url': 'https://evehiclesdb.com/brands/citroen?category=electric-cars'},
 {'ev make': 'CUPRA (4)',
  'url': 'https://evehiclesdb.com/brands/cupra?category=electric-cars'},
 {'ev make': 'Dacia (1)',
  'url': 'https://evehiclesdb.com/brands/dacia?category=electric-cars'},
 {'ev make': 'DS (1)',
  'url': 'https://evehiclesdb.com/brands/ds?category=electric-cars'},
 {'

In [19]:
ev_model = [] #list for all the EV models

for model in ev_makes:
    driver.get(model['url'])
    
    ev_models = driver.find_elements(By.CLASS_NAME, "top_cars_list_item")
    
       
    for models in ev_models:
        models_url = 'https://evehiclesdb.com/transport/'+ model +'/'
        ev_model.append({'ev make': model['ev make'], 'model': models.text, 'url': models_url})

In [44]:
ev_df = pd.DataFrame(ev_model)
ev_df['url string'] = ev_df['model'].str.split(' ').str.join("-").str.lower()
#quick comment a section of code with CTRL + /

print(ev_df)


              ev make                                   model model url                              url string
0          Aiways (1)                               AIWAYS U5      None                               aiways-u5
1    Aston Martin (1)                   ASTON MARTIN RAPIDE E      None                   aston-martin-rapide-e
2           Audi (13)             AUDI Q4 SPORTBACK E-TRON 35      None             audi-q4-sportback-e-tron-35
3           Audi (13)               AUDI Q4 E-TRON 50 QUATTRO      None               audi-q4-e-tron-50-quattro
4           Audi (13)               AUDI Q4 E-TRON 45 QUATTRO      None               audi-q4-e-tron-45-quattro
5           Audi (13)             AUDI Q4 SPORTBACK E-TRON 40      None             audi-q4-sportback-e-tron-40
6           Audi (13)                       AUDI Q4 E-TRON 35      None                       audi-q4-e-tron-35
7           Audi (13)        AUDI E-TRON SPORTBACK 50 QUATTRO      None        audi-e-tron-sportback-50-